### MODEL GRAPHING & ANALYSIS

### GRAPHING FUNCTIONS

In [5]:
def visualize(header, dataframe):
    
    # COLOR SELECTIONS
    colors = ['#ff6666', '#1c8c44', '#5e5ba4', '#8b3c9d', '#767468', '#ffc500', '#ff9400']
    
    # PLACEHOLDER CONTAINER
    structure = []
    
    # CREATE LINE FOR EACH COLUMN IN THE DATAFRAME
    for index, name in enumerate(dataframe.columns):
        structure.append(go.Scatter(
            mode='lines',
            x=dataframe.index,
            y=dataframe[name],
            line=dict(width=1),
            marker=dict(color = colors[index]),
            opacity=0.8,
            yaxis = 'y2',
            name = name
        ))

    # LAYOUT PARAMS
    layout = go.Layout(
        yaxis = dict(domain = [0, 0.2], showticklabels = False),
        yaxis2 = dict(domain = [0.2, 1.0]),
        title = header
    )
    
    # CREATE THE FIGURE
    fig = go.Figure(data = structure, layout = layout)
    
    # ADD BUTTON MAP
    fig.update_layout(
        xaxis=dict(
            rangeselector=dict(
                buttons=list([
                    dict(count=1,
                         label="1m",
                         step="month",
                         stepmode="backward"),
                    dict(count=6,
                         label="6m",
                         step="month",
                         stepmode="backward"),
                    dict(count=1,
                         label="YTD",
                         step="year",
                         stepmode="todate"),
                    dict(count=1,
                         label="1y",
                         step="year",
                         stepmode="backward"),
                    dict(step="all")
                ])
            ),
            rangeslider=dict(
                visible=True
            ),
            type="date"
        )
    )
    
    # FINALLY SHOW THE GRAPH
    fig.show()

In [7]:
def visualize_score(data):
    
    # PLOT THE GRAPH
    data.plot(kind='line', figsize=(16, 4))
    plt.grid(axis='both')

    # SHOW IT
    plt.show()

In [6]:
def graph_fitting(model):
    
    # TRAINING EPOCHS
    epochs = range(len(lstm_one['fitting'].history['loss']))
    
    # CREATE THE FIGURE
    plt.figure(figsize=(16, 4))
    
    # PLOT TRAINING & VALIDATION LOSSES
    plt.plot(epochs, lstm_one['fitting'].history['loss'], 'b', label='Training loss')
    plt.plot(epochs, lstm_one['fitting'].history['val_loss'], 'r', label='Validation loss')
    
    # ADD LEGEND & SHOW
    plt.legend()
    plt.show()

### SCORE FUNCTIONS

In [1]:
def rmse(models):
    
    # PLACEHOLDER
    container = []
    
    # LOOP THROUGH EACH RESULT
    for model in models:
        
        # CALCULATE R2 SCORE & APPEND IT TO THE CONTAINER
        score = r2_score(model['dataset']['test']['labels'], model['predictions'])
        container.append(score)
        
    return container

In [2]:
def mse(models):
    
    # PLACEHOLDER
    container = []
    
    # LOOP THROUGH EACH RESULT
    for model in models:
        
        # CALCULATE R2 SCORE & APPEND IT TO THE CONTAINER
        score = mean_squared_error(model['dataset']['test']['labels'], model['predictions'])
        container.append(score)
        
    return container

In [3]:
def hit_ratios(models):
    
    # PLACEHOLDER
    container = []
    
    # LOOP THROUGH EACH MODEL
    for model in models:
    
        # GENERATE A USABLE DATAFRAME
        dataframe = result_table({
            'base': type_one,
            'predictions': model['predictions'],
            'header': 'Prediction'
        })

        # GENERATE PI SERIES
        pi = generate_pi(dataframe)
        
        # CALCULATE HIT RATIO
        hit = (pi.sum()) / len(pi)
        
        # APPEND TO CONTAINER
        container.append(hit)
    
    return container

In [4]:
def generate_pi(dataframe):
    
    # CREATE A TIMESERIES
    series = (dataframe['Label'].shift(1) - dataframe['Label']) * (dataframe['Prediction'].shift(1) - dataframe['Prediction'])
    
    # RUN THROUGH EACH VALUE & EVALUATE ACCORDINGLY
    series.where(series < 0, 1, inplace=True)
    series.where(series > 0, 0, inplace=True)
    
    return series